In [3]:
"""
Screening code (Python) for threshold/standard-based indicator papers

What it does
1) Loads the dataset.
2) Builds a searchable text field from Title/Abstract/Keywords (robust to column name variants).
3) Tags each record into 5 domains:
   (1) morphology-density
   (2) accessibility-mobility
   (3) green & blue infrastructure
   (4) land use mix
   (5) climate & environmental performance
4) Detects whether the paper likely contains explicit indicator thresholds/standards
   (e.g., “threshold”, “cut-off”, “benchmark”, “WHO”, “≤ 800 m”, “FAR”, “NDVI ≥ 0.3”, etc.)
5) Produces:
   - an Excel file with flags, scores, and “hits” (matched terms)
   - per-domain subsets (optional)

Notes
- This is a metadata pre-screening (Title/Abstract/Keywords). You have to validate thresholds in full text.
- You can tighten/loosen the lexicon below to match your field.
"""

from __future__ import annotations

import re
from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import pandas as pd


# ----------------------------
# 0) Paths
# ----------------------------
from pathlib import Path

# ----------------------------
# 0) Paths (LOCAL MACHINE)
# ----------------------------
INPUT_XLSX = Path(
    "/Users/dwiprabowo/Downloads/greencompactcityurban/combined_wos_scopus.xlsx"
)

OUT_DIR = Path(
    "/Users/dwiprabowo/Downloads/greencompactcityurban/screening_output"
)

OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_ALL = OUT_DIR / "screened_combined_wos_scopus_threshold_domains.xlsx"


# ----------------------------
# 1) Helpers
# ----------------------------
def _normalize_text(x: object) -> str:
    if pd.isna(x):
        return ""
    s = str(x)
    s = s.replace("\n", " ").replace("\r", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s


def _find_first_existing_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """
    Find first column whose normalized name matches any candidate.
    Candidates should be lowercased.
    """
    cols_norm = {c: re.sub(r"[^a-z0-9_]+", "", c.lower()) for c in df.columns}
    cand_norm = [re.sub(r"[^a-z0-9_]+", "", c.lower()) for c in candidates]
    for c, cn in cols_norm.items():
        if cn in cand_norm:
            return c
    return None


def _regex_findall(pattern: re.Pattern, text: str) -> List[str]:
    return list({m.group(0) for m in pattern.finditer(text)})  # unique


def _compile_any(phrases: List[str], flags=re.IGNORECASE) -> re.Pattern:
    # Escape phrases unless they already look like regex (contain \b, [], (), etc.)
    parts = []
    for p in phrases:
        if any(ch in p for ch in [r"\b", "[", "]", "(", ")", "|", "+", "*", "?", "{", "}", "^", "$"]):
            parts.append(p)
        else:
            parts.append(re.escape(p))
    return re.compile(r"(" + r"|".join(parts) + r")", flags)


# ----------------------------
# 2) Lexicon (EDIT HERE)
# ----------------------------
DOMAIN_TERMS: Dict[str, List[str]] = {
    # 1) Morphology–Density
    "morphology_density": [
        r"\bcompact city\b", r"\burban form\b", r"\bmorpholog(y|ical)\b", r"\bdensit(y|ies)\b",
        r"\bfloor area ratio\b", r"\bFAR\b", r"\bFSI\b", r"\bplot ratio\b", r"\bGSI\b", r"\bBCR\b",
        r"\bbuilding height\b", r"\bhigh[- ]rise\b", r"\bmid[- ]rise\b", r"\blow[- ]rise\b",
        r"\bdwelling(s)? per hectare\b", r"\bDU/?ha\b", r"\bpopulation density\b", r"\bnet density\b",
        r"\bintersection density\b", r"\bcompactness index\b", r"\bsprawl\b", r"\bfragmentation\b",
    ],

    # 2) Accessibility–Mobility
    "accessibility_mobility": [
        r"\baccessibilit(y|ies)\b", r"\bwalkabilit(y|ies)\b", r"\bwalkable\b", r"\bwalk[- ]?shed\b",
        r"\bpublic transport\b", r"\btransit\b", r"\bPTAL\b", r"\bcatchment\b", r"\bservice area\b",
        r"\btravel time\b", r"\bcommut(e|ing)\b", r"\bmodal share\b", r"\bactive travel\b",
        r"\bconnectivit(y|ies)\b", r"\bstreet network\b", r"\bintersection\b", r"\b15[- ]minute city\b",
        r"\b400\s?m\b", r"\b800\s?m\b", r"\b10\s?min(ute)?\b", r"\b15\s?min(ute)?\b",
    ],

    # 3) Green & Blue Infrastructure
    "green_blue_infra": [
        r"\bgreen space(s)?\b", r"\burban green\b", r"\burban park(s)?\b", r"\bgreen infrastruct(ure|ures)\b",
        r"\bblue[- ]green\b", r"\bblue space(s)?\b", r"\bwater bod(y|ies)\b", r"\briver(s)?\b",
        r"\blake(s)?\b", r"\bwetland(s)?\b", r"\bcoast(al)?\b", r"\bcanopy\b", r"\btrees?\b",
        r"\bNDVI\b", r"\bEVI\b", r"\bvegetation\b", r"\becosystem service(s)?\b",
        r"\bper capita green\b", r"\bm2\/capita\b", r"\bm²\/capita\b",
    ],

    # 4) Land Use Mix
    "land_use_mix": [
        r"\bland use\b", r"\bland[- ]use\b", r"\bmixed[- ]use\b", r"\bland use mix\b",
        r"\bentropy\b", r"\bShannon\b", r"\bdiversit(y|ies)\b", r"\bfunctional mix\b",
        r"\bMXI\b", r"\bjob[- ]housing balance\b", r"\bLUM\b", r"\bzoning\b",
        r"\bresidential\b", r"\bcommercial\b", r"\bindustrial\b", r"\bamenit(y|ies)\b",
    ],

    # 5) Climate & Environmental Performance
    "climate_env_performance": [
        r"\burban heat island\b", r"\bUHI\b", r"\bland surface temperature\b", r"\bLST\b",
        r"\blocal climate zone\b", r"\bLCZ\b", r"\bthermal\b", r"\bheat stress\b",
        r"\bmitigation\b", r"\bcooling\b", r"\bcooling effect\b", r"\benergy efficien(t|cy)\b",
        r"\benergy consumption\b", r"\bemission(s)?\b", r"\bcarbon\b", r"\bCO2\b",
        r"\bPM2\.5\b", r"\bair quality\b", r"\bflood\b", r"\bstormwater\b", r"\bresilien(ce|t)\b",
        r"\bperformance\b", r"\benvironmental performance\b",
    ],
}

# “Threshold / Standard” detectors (you can add local policy names, regs, codes)
THRESHOLD_TERMS = [
    r"\bthreshold(s)?\b", r"\bcut[- ]?off\b", r"\bcutoff\b", r"\bbenchmark(s)?\b",
    r"\bstandard(s)?\b", r"\bguideline(s)?\b", r"\bcriteria\b", r"\bnorm(s)?\b",
    r"\brecommended\b", r"\btarget(s)?\b", r"\bminimum\b", r"\bmaximum\b",
    r"\blimit(s)?\b", r"\brange\b", r"\bclassified as\b", r"\bclassification\b",
    r"\bWHO\b", r"\bUN[- ]?Habitat\b", r"\bLEED\b", r"\bBREEAM\b",
    r"\bISO\b", r"\bEN\s?\d+\b",
]

# Numeric inequality / threshold patterns (captures things like ">= 0.3", "≤800 m", "FAR > 2.0")
NUMERIC_THRESHOLD_PATTERNS = [
    r"(>=|≤|>=|=>|>|<|≤=|=<)\s*\d+(\.\d+)?\s*(%|m|km|ha|m2|m²|°c|c|k|w\/m2|w\/m²|mj|kwh|kWh|du\/ha)?\b",
    r"\b\d+(\.\d+)?\s*(%|m|km|min|minute|minutes|ha|m2|m²|°c|°C|kwh|kWh)\s*(or\s*less|or\s*more|maximum|minimum)\b",
    r"\bwithin\s+\d+(\.\d+)?\s*(m|km|min|minutes)\b",
]

# Optional: method cues that often co-occur with operational thresholds (not required)
METHOD_CUES = [
    r"\bindex\b", r"\bindicator(s)?\b", r"\bscor(e|ing)\b", r"\bcomposite\b",
    r"\bGIS\b", r"\bremote sensing\b", r"\bsatellite\b", r"\bspatial metric(s)?\b",
    r"\bmachine learning\b", r"\bdeep learning\b", r"\brandom forest\b",
]


# Compile regex
DOMAIN_RE = {k: _compile_any(v) for k, v in DOMAIN_TERMS.items()}
THRESHOLD_RE = _compile_any(THRESHOLD_TERMS)
METHOD_RE = _compile_any(METHOD_CUES)
NUMERIC_RE_LIST = [re.compile(p, re.IGNORECASE) for p in NUMERIC_THRESHOLD_PATTERNS]


# ----------------------------
# 3) Load dataset
# ----------------------------
df = pd.read_excel(INPUT_XLSX)

# Identify main metadata columns robustly
title_col = _find_first_existing_col(df, ["ti", "title"])
abs_col = _find_first_existing_col(df, ["ab", "abstract"])
# Keywords can be in several bibliometrix fields
kw_cols = []
for c in df.columns:
    cn = re.sub(r"[^a-z0-9_]+", "", c.lower())
    if cn in {"de", "id", "keywords", "authorkeywords", "keywordplus", "kw", "kw_merged", "kwmerged"}:
        kw_cols.append(c)

doi_col = _find_first_existing_col(df, ["di", "doi"])

# Create searchable text
df["_title"] = df[title_col].map(_normalize_text) if title_col else ""
df["_abstract"] = df[abs_col].map(_normalize_text) if abs_col else ""
if kw_cols:
    df["_keywords"] = df[kw_cols].astype(str).fillna("").agg(" ; ".join, axis=1).map(_normalize_text)
else:
    df["_keywords"] = ""

df["_search_text"] = (
    df["_title"].str.lower() + " " + df["_abstract"].str.lower() + " " + df["_keywords"].str.lower()
)


# ----------------------------
# 4) Tag domains + threshold evidence
# ----------------------------
def detect_numeric_thresholds(text: str) -> List[str]:
    hits = []
    for rnum in NUMERIC_RE_LIST:
        hits.extend(_regex_findall(rnum, text))
    return list({h.strip() for h in hits if h.strip()})


def detect_hits(pattern: re.Pattern, text: str) -> List[str]:
    return _regex_findall(pattern, text)


# Domain flags + hits
for domain, pat in DOMAIN_RE.items():
    df[f"flag_{domain}"] = df["_search_text"].apply(lambda t: bool(pat.search(t)))
    df[f"hits_{domain}"] = df["_search_text"].apply(lambda t: "; ".join(detect_hits(pat, t))[:2000])

# Threshold evidence
df["flag_threshold_terms"] = df["_search_text"].apply(lambda t: bool(THRESHOLD_RE.search(t)))
df["hits_threshold_terms"] = df["_search_text"].apply(lambda t: "; ".join(detect_hits(THRESHOLD_RE, t))[:2000])

df["hits_numeric_thresholds"] = df["_search_text"].apply(lambda t: "; ".join(detect_numeric_thresholds(t))[:2000])
df["flag_numeric_thresholds"] = df["hits_numeric_thresholds"].str.len().gt(0)

# Method cue (optional)
df["flag_method_cues"] = df["_search_text"].apply(lambda t: bool(METHOD_RE.search(t)))
df["hits_method_cues"] = df["_search_text"].apply(lambda t: "; ".join(detect_hits(METHOD_RE, t))[:2000])

# A simple “screening score”
domain_flags = [c for c in df.columns if c.startswith("flag_") and c not in {"flag_threshold_terms", "flag_numeric_thresholds", "flag_method_cues"}]
df["n_domains_hit"] = df[domain_flags].sum(axis=1)

# Core screening rule (EDIT):
# - Must hit >=1 domain AND (threshold_terms OR numeric_thresholds)
df["screen_pass_threshold_based"] = (df["n_domains_hit"] >= 1) & (df["flag_threshold_terms"] | df["flag_numeric_thresholds"])

# Optional stricter rule:
# - Must hit >=1 domain AND numeric threshold evidence (e.g., >=, ≤, “within 400 m”)
df["screen_pass_numeric_only"] = (df["n_domains_hit"] >= 1) & (df["flag_numeric_thresholds"])

# A combined priority score to sort candidates:
# +3 if numeric thresholds present
# +2 if threshold terms present
# +1 if method cues present
# + (domains hit)
df["priority_score"] = (
    df["n_domains_hit"].astype(int)
    + 3 * df["flag_numeric_thresholds"].astype(int)
    + 2 * df["flag_threshold_terms"].astype(int)
    + 1 * df["flag_method_cues"].astype(int)
)

# Keep a compact view for review
keep_cols = []
for c in [doi_col, title_col, abs_col] + kw_cols:
    if c and c in df.columns and c not in keep_cols:
        keep_cols.append(c)

keep_cols += (
    domain_flags
    + ["n_domains_hit", "flag_threshold_terms", "flag_numeric_thresholds", "flag_method_cues",
       "screen_pass_threshold_based", "screen_pass_numeric_only", "priority_score"]
    + [f"hits_{d}" for d in DOMAIN_RE.keys()]
    + ["hits_threshold_terms", "hits_numeric_thresholds", "hits_method_cues"]
)

out_df = df[keep_cols].copy()

# Sort for manual screening workflow
out_df = out_df.sort_values(by=["screen_pass_threshold_based", "priority_score"], ascending=[False, False])


# ----------------------------
# 5) Export outputs
# ----------------------------
out_df.to_excel(OUT_ALL, index=False)

# Optional: per-domain subsets for convenience
for domain in DOMAIN_RE.keys():
    subset = out_df[
        (df[f"flag_{domain}"]) & (df["screen_pass_threshold_based"])
    ].copy()
    subset_path = OUT_DIR / f"subset_{domain}_threshold_based.xlsx"
    subset.to_excel(subset_path, index=False)

# Summary counts
summary = {
    "total_records": len(df),
    "pass_threshold_based": int(df["screen_pass_threshold_based"].sum()),
    "pass_numeric_only": int(df["screen_pass_numeric_only"].sum()),
}
for domain in DOMAIN_RE.keys():
    summary[f"pass_{domain}_threshold_based"] = int(((df[f"flag_{domain}"]) & (df["screen_pass_threshold_based"])).sum())

summary_df = pd.DataFrame([summary])
summary_path = OUT_DIR / "screening_summary_counts.xlsx"
summary_df.to_excel(summary_path, index=False)

print("DONE")
print(f"- Main output: {OUT_ALL}")
print(f"- Summary:     {summary_path}")
print(f"- Per-domain subsets saved to: {OUT_DIR}")


"""
After running:
1) Open screened_combined_wos_scopus_threshold_domains.xlsx
2) Filter screen_pass_threshold_based = TRUE
3) Sort by priority_score
4) Manually confirm in PDF/full text that thresholds are explicitly stated and extract:
   - indicator name, unit, threshold value, source (WHO/plan code/etc.), scale, context
"""


/var/folders/5t/y04tjy6s14j73msn7v4_djm00000gn/T/ipykernel_70472/1285696963.py:289: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset = out_df[
/var/folders/5t/y04tjy6s14j73msn7v4_djm00000gn/T/ipykernel_70472/1285696963.py:289: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset = out_df[
/var/folders/5t/y04tjy6s14j73msn7v4_djm00000gn/T/ipykernel_70472/1285696963.py:289: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset = out_df[
/var/folders/5t/y04tjy6s14j73msn7v4_djm00000gn/T/ipykernel_70472/1285696963.py:289: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset = out_df[
/var/folders/5t/y04tjy6s14j73msn7v4_djm00000gn/T/ipykernel_70472/1285696963.py:289: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset = out_df[


DONE
- Main output: /Users/dwiprabowo/Downloads/greencompactcityurban/screening_output/screened_combined_wos_scopus_threshold_domains.xlsx
- Summary:     /Users/dwiprabowo/Downloads/greencompactcityurban/screening_output/screening_summary_counts.xlsx
- Per-domain subsets saved to: /Users/dwiprabowo/Downloads/greencompactcityurban/screening_output


'\nAfter running:\n1) Open screened_combined_wos_scopus_threshold_domains.xlsx\n2) Filter screen_pass_threshold_based = TRUE\n3) Sort by priority_score\n4) Manually confirm in PDF/full text that thresholds are explicitly stated and extract:\n   - indicator name, unit, threshold value, source (WHO/plan code/etc.), scale, context\n'